In [1]:
import customtkinter as ctk
import tkinter as tk
import csv
import os
from tkinter import ttk
class CSVViewer:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("CSV Viewer")
        self.root.geometry("800x600")

        # Create a label and entry for the file path
        self.file_path_label = tk.Label(self.root, text="File Path:")
        self.file_path_label.pack()
        self.file_path_entry = tk.Entry(self.root, width=50)
        self.file_path_entry.pack()

        # Create a button to browse for the file
        self.browse_button = tk.Button(self.root, text="Browse", command=self.browse_file)
        self.browse_button.pack()

        # Create a Treeview widget to display the CSV data
        self.tree = ttk.Treeview(self.root)
        self.tree.pack()

        # Create a column for the treeview
        self.tree["columns"] = ("Title", "Author", "Category", "Year","Rating")
        for column in self.tree["columns"]:
            self.tree.column(column, width=100)
            self.tree.heading(column, text=column)

        # Create a dropdown menu for sorting options
        self.sort_by_label = tk.Label(self.root, text="Sort by:")
        self.sort_by_label.pack()
        self.sort_by_var = tk.StringVar()
        self.sort_by_var.set("None")
        self.sort_by_menu = tk.OptionMenu(self.root, self.sort_by_var, "None","Title", "Author", "Category", "Year","Rating")
        self.sort_by_menu.pack()

        # Create a button to sort the data
        self.sort_button = tk.Button(self.root, text="Sort", command=self.sort_data)
        self.sort_button.pack()
        self.browse_file()
        self.sort_data()
    def browse_file(self):
        # Open a file dialog to select the CSV file
        #file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        file_path = "top_100_books_final.csv"
        if file_path:
            self.file_path_entry.delete(0, tk.END)
            self.file_path_entry.insert(0, file_path)

    def sort_data(self):
        # Get the file path and sorting option from the GUI
        file_path = self.file_path_entry.get()
        sort_by = self.sort_by_var.get()

        # Read the CSV file
        with open(file_path, "r") as file:
            reader = csv.reader(file)
            data = list(reader)

        # Sort the data based on the selected column
        if sort_by == "Title":
            data.sort(key=lambda x: x[0])
        elif sort_by == "Author":
            data.sort(key=lambda x: x[1])
        elif sort_by == "Category":
            data.sort(key=lambda x: x[2])
        elif sort_by == "Year":
            data.sort(key=lambda x: x[3])
        elif sort_by == "Rating":
            data.sort(key=lambda x: x[4])

        # Clear the treeview and display the sorted data
        self.tree.delete(*self.tree.get_children())
        for row in data:
            self.tree.insert("", "end", values=row)

    def run(self):
        self.root.mainloop()
# Book class
class Book:
    def __init__(self, title, author, genre, year,rating = ""):
        self.title = title
        self.author = author
        self.genre = genre
        self.year = year
        self.rating = rating

# Library class
class Library:
    def __init__(self, filename="top_100_books_final.csv"):
        self.filename = filename
        self.books = self.load_books()

    def add_book(self, book):
        self.books.append(book)
        self.save_books()

    def remove_book(self, title):
        self.books = [book for book in self.books if book.title.lower() != title.lower()]
        self.save_books()

    def search_books(self, query):
        query_lower = query.lower()
        query_words = set(query_lower.split())
        
        def matches(book):
            title_words = set(book.title.lower().split())
            author_words = set(book.author.lower().split())
            return query_lower in book.title.lower() or query_lower in book.author.lower() or not query_words.isdisjoint(title_words) or not query_words.isdisjoint(author_words)
        
        return [book for book in self.books if matches(book)]

    def save_books(self):
        with open(self.filename, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["Title", "Author", "Genre", "Year","Rating"])
            for book in self.books:
                writer.writerow([book.title, book.author, book.genre, book.year,book.rating])

    def load_books(self):
        books = []
        if os.path.exists(self.filename):
            with open(self.filename, "r") as f:
                reader = csv.reader(f)
                next(reader, None)  # Skip header
                for row in reader:
                    if row:
                        books.append(Book(*row))
        return books

# GUI Application
class LibraryApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Library Management System")
        self.geometry("900x650")
        self.configure(padx=20, pady=20)
        
        self.library = Library()
        
        # Widgets
        self.frame = ctk.CTkFrame(self)
        self.frame.pack(fill="x", pady=10)
        
        self.title_label = ctk.CTkLabel(self.frame, text="Title:")
        self.title_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.title_entry = ctk.CTkEntry(self.frame, width=200)
        self.title_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        
        self.author_label = ctk.CTkLabel(self.frame, text="Author:")
        self.author_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.author_entry = ctk.CTkEntry(self.frame, width=200)
        self.author_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        
        self.genre_label = ctk.CTkLabel(self.frame, text="Genre:")
        self.genre_label.grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.genre_entry = ctk.CTkEntry(self.frame, width=200)
        self.genre_entry.grid(row=2, column=1, padx=5, pady=5, sticky="ew")
        
        self.year_label = ctk.CTkLabel(self.frame, text="Year:")
        self.year_label.grid(row=3, column=0, padx=5, pady=5, sticky="w")
        self.year_entry = ctk.CTkEntry(self.frame, width=200)
        self.year_entry.grid(row=3, column=1, padx=5, pady=5, sticky="ew")
        
        self.add_button = ctk.CTkButton(self.frame, text="Add Book", command=self.add_book)
        self.add_button.grid(row=4, column=0, columnspan=2, pady=10)

        self.search_label = ctk.CTkLabel(self, text="Enter your keyword down here:")
        self.search_label.pack(pady=1)
        
        self.search_entry = ctk.CTkEntry(self, width=250)
        self.search_entry.pack(pady=5)
        self.search_button = ctk.CTkButton(self, text="Search Book", command=self.search_book)
        self.search_button.pack(pady=5)
        
        self.delete_button = ctk.CTkButton(self, text="Delete Book", command=self.delete_book)
        self.delete_button.pack(pady=5)
        
        self.show_all_button = ctk.CTkButton(self, text="Show All Books", command=self.show_all_sorted)
        self.show_all_button.pack(pady=5)
        
        self.tree = ttk.Treeview(self, columns=("Title", "Author", "Genre", "Year","Rating"), show="headings")
        self.tree.heading("Title", text="Title",anchor ='c')
        self.tree.heading("Author", text="Author",anchor ='c')
        self.tree.heading("Genre", text="Genre")
        self.tree.heading("Year", text="Year")
        self.tree.heading("Rating", text="Rating")
        self.tree.pack(pady=10, fill="both", expand=True)
        self.display_books()
        self.sort_data()

    def sort_data(self):
        # Get the file path and sorting option from the GUI
        file_path = "top_100_books_final.csv"
        #sort_by = self.sort_by_var.get()
        sort_by = "Rating"
        # Read the CSV file
        with open(file_path, "r") as file:
            reader = csv.reader(file)
            data = list(reader)

        # Sort the data based on the selected column
        if sort_by == "Title":
            data.sort(key=lambda x: x[0])
        elif sort_by == "Author":
            data.sort(key=lambda x: x[1])
        elif sort_by == "Category":
            data.sort(key=lambda x: x[2])
        elif sort_by == "Year":
            data.sort(key=lambda x: x[3])
        elif sort_by == "Rating":
            data.sort(key=lambda x: x[4])

        # Clear the treeview and display the sorted data
        self.tree.delete(*self.tree.get_children())
        for row in data:
            self.tree.insert("", "end", values=row)
    def show_all_sorted(self):
        app = CSVViewer()
        app.run()
    def add_book(self):
        book = Book(
            self.title_entry.get(),
            self.author_entry.get(),
            self.genre_entry.get(),
            self.year_entry.get()
        )
        self.library.add_book(book)
        self.display_books()
        
    def search_book(self):
        query = self.search_entry.get()
        results = self.library.search_books(query)
        self.tree.delete(*self.tree.get_children())
        for book in results:
            self.tree.insert("", "end", values=(book.title, book.author, book.genre, book.year, book.rating))
        
    def delete_book(self):
        title = self.title_entry.get()
        self.library.remove_book(title)
        self.display_books()
        
    def show_all_books(self):
        self.display_books()
        
    def display_books(self):
        self.tree.delete(*self.tree.get_children())
        for book in self.library.books:
            self.tree.insert("", "end", values=(book.title, book.author, book.genre, book.year,book.rating))

if __name__ == "__main__":
    app = LibraryApp()
    app.mainloop()